In [1]:
import zipfile
import pandas as pd 
import numpy as np 

zip_path='../../DATAFLOW_2026_UET.EPOCH_0_AUTOSCALING_ANALYSIS/data/raw/DATA-20260121T113005Z-1-001.zip'
def load_zip_file(path):
    with zipfile.ZipFile (zip_path,'r') as z :
        with z.open(path,'r') as f:
            return f.read().decode('utf-8')
        

In [2]:
train=load_zip_file(r'DATA/train.txt')
test=load_zip_file(r'DATA/test.txt')

In [3]:
lines_train=train.splitlines()
lines_test=test.splitlines()
print(lines_train[0])
print(lines_test[0])

199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245
ix-mia1-02.ix.netcom.com - - [23/Aug/1995:00:00:00 -0400] "GET /ksc.html HTTP/1.0" 200 7087


In [4]:
import re

pattern = re.compile(
    r'(?P<host>\S+) .* '
    r'\[(?P<timestamp>[^\]]+)\] '
    r'"(?P<request>[^"]+)" '
    r'(?P<response>\d{3}) '
    r'(?P<bytes>\d+|-)'
)

In [5]:
def to_dataframe(lines):
    rows = []
    for line in lines:
        m = pattern.search(line)
        if m:
            row = m.groupdict()
            row["bytes"] = 0 if row["bytes"] == "-" else int(row["bytes"])
            rows.append(row)

    return pd.DataFrame(rows)

In [6]:
train_df=to_dataframe(lines_train)
test_df=to_dataframe(lines_test)

In [7]:
train_df.head(5)

,host,timestamp,request,response,bytes
0,199.72.81.55,01/Jul/1995:00:00:01 -0400,GET /history/apollo/ HTTP/1.0,200,6245
1,unicomp6.unicomp.net,01/Jul/1995:00:00:06 -0400,GET /shuttle/countdown/ HTTP/1.0,200,3985
2,199.120.110.21,01/Jul/1995:00:00:09 -0400,GET /shuttle/missions/sts-73/mission-sts-73.ht...,200,4085
3,burger.letters.com,01/Jul/1995:00:00:11 -0400,GET /shuttle/countdown/liftoff.html HTTP/1.0,304,0
4,199.120.110.21,01/Jul/1995:00:00:11 -0400,GET /shuttle/missions/sts-73/sts-73-patch-smal...,200,4179


In [8]:
test_df.head(5)

,host,timestamp,request,response,bytes
0,ix-mia1-02.ix.netcom.com,23/Aug/1995:00:00:00 -0400,GET /ksc.html HTTP/1.0,200,7087
1,internet-gw.watson.ibm.com,23/Aug/1995:00:00:05 -0400,GET /history/apollo/pad-abort-test-2/pad-abort...,200,1292
2,ix-mia1-02.ix.netcom.com,23/Aug/1995:00:00:06 -0400,GET /images/ksclogo-medium.gif HTTP/1.0,200,5866
3,internet-gw.watson.ibm.com,23/Aug/1995:00:00:08 -0400,GET /history/apollo/pad-abort-test-2/pad-abort...,200,1625
4,internet-gw.watson.ibm.com,23/Aug/1995:00:00:10 -0400,GET /history/apollo/pad-abort-test-2/pad-abort...,404,0


In [9]:
train_df.to_csv("../../DATAFLOW_2026_UET.EPOCH_0_AUTOSCALING_ANALYSIS/data/processed/train.csv")
test_df.to_csv("../../DATAFLOW_2026_UET.EPOCH_0_AUTOSCALING_ANALYSIS/data/processed/test.csv")

In [10]:
with zipfile.ZipFile("../../DATAFLOW_2026_UET.EPOCH_0_AUTOSCALING_ANALYSIS/data/processed/data.zip", "w", zipfile.ZIP_DEFLATED) as z:
    z.write("../../DATAFLOW_2026_UET.EPOCH_0_AUTOSCALING_ANALYSIS/data/processed/train.csv", arcname="train.csv")
    z.write("../../DATAFLOW_2026_UET.EPOCH_0_AUTOSCALING_ANALYSIS/data/processed/test.csv", arcname="test.csv")